In [247]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [248]:
import io
import pprint

import pymongo
import pandas as pd
from IPython import display

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

from climatools.cliradlw.dataset import load_output_file
import climatools.html.html as climahtml
from climatools.plot.plot import *

In [249]:
output_notebook()

Loading BokehJS ...

In [250]:
client = pymongo.MongoClient('localhost', 27017)

In [251]:
db = client.lblnew

In [252]:
collection = db['bestfit_lw']

In [253]:
MOLECULE, BAND = 'co2', '3b'

COMMITNUMBER = 'a22ab94'

CONCS = [400e-6, 800e-6]

ATMPRO = 'mls'

In [267]:
all([True, 6, None])

False

In [276]:
def get_queries():
    return [{'param.molecule': MOLECULE,
             'param.band': BAND,
             'param.commitnumber': COMMITNUMBER,
             'param.conc': conc,
             'param.atmpro': ATMPRO}
            for conc in CONCS]



def get_docs():
    queries = get_queries()
    rs = [collection.find_one(query) for query in qs]
    
    try:
        assert all(rs)
    except:
        notfounds = [q for q, r in zip(qs, rs) if r == None]
        print('No results were found for the following queries:')
        pprint.pprint(notfounds)
        proceed = input('Continue only with those found? (yes/no)')
        if proceed == 'yes':
            pass
        else:
            raise
            
    return [r for r in rs if r]

    
    for query in queries:
        r = collection.find_one(query) 
        if r == None:
            print('No results were found for the query:')
            pprint.pprint(query)
            


get_docs()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [254]:
query = {'param.molecule': MOLECULE,
         'param.band': BAND,
         'param.commitnumber': COMMITNUMBER,
         'param.conc': CONCS[0],
         'param.atmpro': ATMPRO}

R1x = collection.find_one(query)

ds_flux_crd = load_output_file(io.StringIO(R1x['output_fluxg']))
ds_cool_crd = load_output_file(io.StringIO(R1x['output_coolrg']))
ds_flux_wgt = load_output_file(io.StringIO(R1x['output_wfluxg']))
ds_cool_wgt = load_output_file(io.StringIO(R1x['output_wcoolrg']))

FLUX_CRD_1x = ds_flux_crd
COOL_CRD_1x = ds_cool_crd
FLUX_WGT_1x = ds_flux_wgt
COOL_WGT_1x = ds_cool_wgt

In [255]:
query = {'param.molecule': MOLECULE,
         'param.band': BAND,
         'param.commitnumber': COMMITNUMBER,
         'param.conc': CONCS[1],
         'param.atmpro': ATMPRO}
R2x = collection.find_one(query)

ds_flux_crd = load_output_file(io.StringIO(R2x['output_fluxg']))
ds_cool_crd = load_output_file(io.StringIO(R2x['output_coolrg']))
ds_flux_wgt = load_output_file(io.StringIO(R2x['output_wfluxg']))
ds_cool_wgt = load_output_file(io.StringIO(R2x['output_wcoolrg']))

FLUX_CRD_2x = ds_flux_crd
COOL_CRD_2x = ds_cool_crd
FLUX_WGT_2x = ds_flux_wgt
COOL_WGT_2x = ds_cool_wgt

In [256]:
def script_print_params():

    labels = [f'conc.={conc}' for conc in CONCS]
    
    params = [r['param'] for r in Rs]
    
    df = pd.DataFrame(columns=labels)
    df.columns.name = 'lblnew-bestfit run'
    df.index.name = 'input parameter'
    for n in params[0].keys():
        for label, param in zip(labels, params):
            df.loc[n, label] = param[n]
    
    display.display(df)   
    display.display(display.Markdown(
        ("TABLE. lblnew-bestfit input parameters of "
         "runs considered in this analysis.")))
    


In [257]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
        
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
            
    return ds['coolrg']


def pltdata_cool(g=None):
    colors = all_palettes['Accent'][7]
    
    diff_crd = COOL_CRD_2x - COOL_CRD_1x
    diff_wgt = COOL_WGT_2x - COOL_WGT_1x
    
    if g == None:
        diff_crd = diff_crd.sum('g')
        diff_wgt = diff_wgt.sum('g')
    else:
        diff_crd = diff_crd.sel(g=g)
        diff_wgt = diff_wgt.sel(g=g)
    
    data = [
        {'label': 'CRD',
         'srs': fmt_cool(diff_crd),
         'line_dash': 'solid', 'line_width': 1.5,
         'marker': 'circle', 'mark_size': 5,
         'color': colors[2], 'alpha': 1},
        {'label': 'WGT igg=1',
         'srs': fmt_cool(diff_wgt.sel(igg=1)),
         'line_dash': 'solid', 'line_width': 3,
         'marker': 'square', 'marker_size': 5,
         'color': colors[3], 'alpha': .6},
        {'label': 'WGT igg=10',
         'srs': fmt_cool(diff_wgt.sel(igg=10)),
         'line_dash': 'solid', 'line_width': 2,
         'marker': 'triangle', 'marker_size': 2,
         'color': colors[1], 'alpha': .3}
    ]
    return data


def plt_cool(pltdata=None):
    '''
    Plot cooling rate profiles on linear
    and log pressure axes.
    
    Parameters
    ----------
    pltdata: dict
        Plot data, including colour, line attributes, etc.
    everything: bokeh.models.layouts.Column
        Cooling rate profile plot.
    '''
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    

    
def script_plt_cool():
    ng = sum(r['param']['ng_refs'])
    for g in range(1, ng + 1):
        data = pltdata_cool(g=g)
        p = plt_cool(pltdata=data)
        show(p)
        display.display(
            display.Markdown(('*FIGURE.* Cooling rate difference. '
                              '(2x conc - 1x conc).  '
                              f'g={g}.')))


In [258]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.squeeze('igg').drop('igg')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def concat_tbs(benchmark, *tbs):
    dfs = [benchmark['df']] + [tb['df'] for tb in tbs]    
    keys = [benchmark['label']] + [tb['label'] for tb in tbs]
    return pd.concat(dfs, axis=1, keys=keys)

def concat_tbdiffs(benchmark, *tbs):
    dfs = [benchmark['df']] + [tb['df'] for tb in tbs]
    dfs = [df - benchmark['df'] for df in dfs]
    keys = [benchmark['label']] + [tb['label'] for tb in tbs]
    keys = [f"{key} - {benchmark['label']}" for key in keys]
    return pd.concat(dfs, axis=1, keys=keys)
    



def tbdata_flux():
    atmpro = 'mls'
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    ilevels = [0, tropopauses[atmpro], -1]
    
    dflux_crd = FLUX_CRD_2x - FLUX_CRD_1x
    dflux_wgt = FLUX_WGT_2x - FLUX_WGT_1x
    
    benchmark = {'label': '(CRD 2x-1x)',
                 'df': fmt_flux(dflux_crd, ilevels=ilevels)}
    
    tbs = [{'label': '(WGT igg=1 2x-1x)',
           'df': fmt_flux(dflux_wgt.sel(igg=[1]), ilevels=ilevels)},
           {'label': '(WGT igg=10 2x-1x)',
            'df': fmt_flux(dflux_wgt.sel(igg=[10]), ilevels=ilevels)}]

    return {'benchmark': benchmark, 'others': tbs}
    

def show_tbs(data=None):
    benchmark = data['benchmark']
    tbs = data['others']
    
    display.display(concat_tbs(benchmark, *tbs))
    display.display(display.Markdown(
        f"*TABLE.* 2x{MOLECULE} - 1x{MOLECULE}, for CRD, igg=1, and igg=10."))
    
    display.display(concat_tbdiffs(benchmark, *tbs))
    display.display(display.Markdown(
        ("*TABLE.* Difference in (2xCO2 - 1xCO2), between "
         "CRD, igg=1, and igg=10.")))
    

def script_flux():
    data = tbdata_flux()
    show_tbs(data=data)

In [259]:
def show_html(s):
    display.display(display.HTML(s))

def show_markdown(s):
    display.display(display.Markdown(s))
    

def script():
    
    title = f'# {MOLECULE} band{BAND} at Different Concentrations'
    s_input = 'Input parameters'
    s_cool = 'Cooling rate difference profiles'
    s_flux = 'Flux difference'
    
    # title
    show_markdown(title)
    
    # TOC
    show_markdown('Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_cool))
    show_html(climahtml.getHTML_hrefanchor(s_flux))
    
    # Input Parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(f'# {s_input}')
    script_print_params()
    
    # Cooling rate difference profiles
    show_html(climahtml.getHTML_idanchor(s_cool))
    show_markdown(f'# {s_cool}')
    script_plt_cool()
    
    # Flux difference profiles
    show_html(climahtml.getHTML_idanchor(s_flux))
    show_markdown(f'# {s_flux}')
    script_flux()
    
    

In [260]:
script()

# co2 band3b at Different Concentrations

Table of Contents

# Input parameters

lblnew-bestfit run,1x conc,2x conc
input parameter,,
atmpro,mls,mls
band,3b,3b
commitnumber,a22ab94,a22ab94
conc,0.0004,0.0008
dv,0.001,0.001
klin,0,0
molecule,co2,co2
ng_adju,"[0, 0]","[0, 0]"
ng_refs,"[5, 2]","[5, 2]"


TABLE. lblnew-bestfit input parameters of runs considered in this analysis.

...

# Flux difference

(CRD 2x-1x)                         (WGT igg=1 2x-1x)  \
                        flug          fldg     fnetg              flug   
g pressure level                                                         
1 0.000    1        0.000206  0.000000e+00  0.000206          0.000264   
  180.875  41       0.000000  0.000000e+00  0.000000          0.000000   
  1013.000 76       0.000000  0.000000e+00  0.000000          0.000000   
2 0.000    1        0.007216  0.000000e+00  0.007216          0.006173   
  180.875  41       0.000004  4.960000e-07  0.000005          0.000000   
  1013.000 76       0.000000  5.990000e-06  0.000006          0.000000   
3 0.000    1        0.035888  0.000000e+00  0.035888          0.054774   
  180.875  41       0.001153 -3.294000e-05  0.001120          0.000000   
  1013.000 76       0.000000  1.077100e-04  0.000108          0.000000   
4 0.000    1       -0.131362  0.000000e+00 -0.131362         -0.139067   
  180.875  41       0.009277  1.830990e-03  0.011108          0.000000   
  1013.000 76       0.000000  8.520000e-04  0.000852          0.000000   
5 0.000    1       -0.378218  0.000000e+00 -0.378218         -0.432800   
  180.875  41       0.043210  3.463480e-02  0.077844          0.000000   
  1013.000 76       0.000000  4.987000e-03  0.004988          0.000000   
6 0.000    1       -0.172365  0.000000e+00 -0.172365         -0.364771   
  180.875  41       0.308203  4.205996e-01  0.728803          0.181547   
  1013.000 76       0.000000  1.660000e-02  0.016600          0.000000   
7 0.000    1        0.573429  0.000000e+00  0.573429          0.611361   
  180.875  41       0.550431  4.092639e-01  0.959695          0.566790   
  1013.000 76       0.000000  1.335752e-01  0.133575          0.000000   

                                     (WGT igg=10 2x-1x)                \
                      fldg     fnetg               flug          fldg   
g pressure level                                                        
1 0.000    1      0.000000  0.000264           0.000269  0.000000e+00   
  180.875  41     0.000000  0.000000           0.000000  0.000000e+00   
  1013.000 76     0.000000  0.000000           0.000000  0.000000e+00   
2 0.000    1      0.000000  0.006173           0.005810  0.000000e+00   
  180.875  41     0.000000  0.000000           0.000000  0.000000e+00   
  1013.000 76     0.000000  0.000000           0.000000  0.000000e+00   
3 0.000    1      0.000000  0.054774           0.060033  0.000000e+00   
  180.875  41     0.000000  0.000000           0.000000  0.000000e+00   
  1013.000 76     0.000000  0.000000           0.000000  0.000000e+00   
4 0.000    1      0.000000 -0.139067          -0.147010  0.000000e+00   
  180.875  41     0.000000  0.000000           0.000000  0.000000e+00   
  1013.000 76     0.000000  0.000000           0.000000  0.000000e+00   
5 0.000    1      0.000000 -0.432800          -0.437048  0.000000e+00   
  180.875  41     0.000011  0.000011           0.000001  6.000000e-07   
  1013.000 76     0.000000  0.000000           0.000000  0.000000e+00   
6 0.000    1      0.000000 -0.364771          -0.375826  0.000000e+00   
  180.875  41     0.103548  0.285095           0.158913  4.883050e-02   
  1013.000 76     0.002005  0.002005           0.000000  5.950000e-04   
7 0.000    1      0.000000  0.611361           0.616057  0.000000e+00   
  180.875  41     0.425853  0.992643           0.562076  4.469342e-01   
  1013.000 76     0.068734  0.068734           0.000000  5.793120e-02   

                            
                     fnetg  
g pressure level            
1 0.000    1      0.000269  
  180.875  41     0.000000  
  1013.000 76     0.000000  
2 0.000    1      0.005810  
  180.875  41     0.000000  
  1013.000 76     0.000000  
3 0.000    1      0.060033  
  180.875  41     0.000000  
  1013.000 76     0.000000  
4 0.000    1     -0.147010  
  180.875  41     0.000000  
  1013.000 76     0.000000  
5 0.000    1     -0.437048  
  180.875  41     0.000

*TABLE.* 2xCO2 - 1xCO2, for CRD, igg=1, and igg=10.

(CRD 2x-1x) - (CRD 2x-1x)             \
                                      flug fldg fnetg   
g pressure level                                        
1 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
2 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
3 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
4 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
5 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
6 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   
7 0.000    1                           0.0  0.0   0.0   
  180.875  41                          0.0  0.0   0.0   
  1013.000 76                          0.0  0.0   0.0   

                 (WGT igg=1 2x-1x) - (CRD 2x-1x)                          \
                                            flug          fldg     fnetg   
g pressure level                                                           
1 0.000    1                            0.000058  0.000000e+00  0.000058   
  180.875  41                           0.000000  0.000000e+00  0.000000   
  1013.000 76                           0.000000  0.000000e+00  0.000000   
2 0.000    1                           -0.001043  0.000000e+00 -0.001043   
  180.875  41                          -0.000004 -4.960000e-07 -0.000005   
  1013.000 76                           0.000000 -5.990000e-06 -0.000006   
3 0.000    1                            0.018886  0.000000e+00  0.018886   
  180.875  41                          -0.001153  3.294000e-05 -0.001120   
  1013.000 76                           0.000000 -1.077100e-04 -0.000108   
4 0.000    1                           -0.007704  0.000000e+00 -0.007704   
  180.875  41                          -0.009277 -1.830990e-03 -0.011108   
  1013.000 76                           0.000000 -8.520000e-04 -0.000852   
5 0.000    1                           -0.054582  0.000000e+00 -0.054582   
  180.875  41                          -0.043210 -3.462370e-02 -0.077833   
  1013.000 76                           0.000000 -4.987000e-03 -0.004988   
6 0.000    1                           -0.192406  0.000000e+00 -0.192406   
  180.875  41                          -0.126656 -3.170515e-01 -0.443708   
  1013.000 76                           0.000000 -1.459500e-02 -0.014595   
7 0.000    1                            0.037932  0.000000e+00  0.037932   
  180.875  41                           0.016359  1.658870e-02  0.032948   
  1013.000 76                           0.000000 -6.484130e-02 -0.064841   

                 (WGT igg=10 2x-1x) - (CRD 2x-1x)                          
                                             flug          fldg     fnetg  
g pressure level                                                           
1 0.000    1                             0.000062  0.000000e+00  0.000062  
  180.875  41                            0.000000  0.000000e+00  0.000000  
  1013.000 76                            0.000000  0.000000e+00  0.000000  
2 0.000    1                            -0.001406  0.000000e+00 -0.001406  
  180.875  41                           -0.000004 -4.960000e-07 -0.000005  
  1013.000 76                            0.000000 -5.990000e-06 -0.000006  
3 0.000    1                             0.024145  0.000000e+00  0.024145  
  180.875  41                           -0.001153  3.294000e-05

*TABLE.* Difference in (2xCO2 - 1xCO2), between CRD, igg=1, and igg=10.

In [261]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
